In [1]:
%load_ext autoreload
%autoreload 2

## Simulate cranial defects

In [1]:
from headctools.tools.offline_augmentation import simulate_cr_images as sim
sim('/home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/test',2,)

Folder: /home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/test
Input: /home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/test/CQ500-CT-370_CT BONE THIN.nii.gz.
  Saved: /home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/test/augm/CQ500-CT-370_CT BONE THIN0_sim.nii.gz.
  Saved: /home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/test/augm/CQ500-CT-370_CT BONE THIN1_sim.nii.gz.
Input: /home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/test/CQ500-CT-356_CT PLAIN THIN.nii.gz.
  Saved: /home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/test/augm/CQ500-CT-356_CT PLAIN THIN0_sim.nii.gz.
  Saved: /home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/test/augm/CQ500-CT-356_CT PLAIN THIN1_sim.nii.gz.
Input: /home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/test/CQ500-CT-469_CT 0.625mm.nii.gz.
  Saved: /home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/test/augm/CQ500-CT-469_CT 0.625mm0_sim.nii.gz.
  Saved: /home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/t

## Test convolutional PCA reconstruction

In [2]:
import os

import torch
import vedo

from models.PCAH import PCAH_Net
from utils.datasetHeads import MeshHeadsDataset

### Load test images

In [3]:
test_head_files = "heads_files/test_images_heads.txt"
images = open(test_head_files, 'r').read().splitlines()
im_path = os.path.abspath('../Chest-xray-landmark-dataset/Images')

dataset = MeshHeadsDataset(images, img_path=im_path)

device = 'cuda:0'

In [4]:
interp_factor = 0.85

config = {
    'device': device,
    'Heads': True,
    'Lungs': False,
    'latents': 60,
    'inputsize': 1024,
    'interpolate': True,
    'h': int(512 * interp_factor),
    'w': int(512 * interp_factor),
    'slices': int(233 * interp_factor),
}

pcaNet = PCAH_Net(config.copy()).to(device)
pcaNet.load_state_dict(torch.load('Training/PCAH/bestMSE.pt'))
pcaNet.eval()
print('Model loaded')

Model loaded


In [5]:
print(f"Test images: {len(dataset)}")

# I can use any img since all .faces() are the same
ex_img = os.path.join(
    im_path, 'CQ500-CT-12_CT Thin Plain_decimated_1perc_dfm.vtk'
)
out_path = os.path.join(im_path, f'pred_PCAH')  # Predictions subfolder
os.makedirs(out_path, exist_ok=True)

faces = vedo.Mesh(ex_img).faces()

for i, sample in enumerate(dataset):
    test_im = sample['image'].unsqueeze(0)
    out_img = pcaNet(test_im.to(device))
    points = out_img.reshape(-1, 3).detach().cpu().numpy()

    out_fpath = os.path.join(out_path, images[i].replace('.nii.gz', '.stl'))

    restored_mesh = vedo.Mesh([points, faces])
    restored_mesh.write(out_fpath)
    print(f'saved mesh {out_fpath}')
    del test_im, out_img


Test images: 36
saved mesh /media/fmatzkin/data/franco/Code/Chest-xray-landmark-dataset/Images/pred_PCAH/CQ500-CT-370_CT BONE THIN.stl
saved mesh /media/fmatzkin/data/franco/Code/Chest-xray-landmark-dataset/Images/pred_PCAH/CQ500-CT-356_CT PLAIN THIN.stl
saved mesh /media/fmatzkin/data/franco/Code/Chest-xray-landmark-dataset/Images/pred_PCAH/CQ500-CT-469_CT 0.625mm.stl
saved mesh /media/fmatzkin/data/franco/Code/Chest-xray-landmark-dataset/Images/pred_PCAH/CQ500-CT-241_CT PLAIN THIN.stl
saved mesh /media/fmatzkin/data/franco/Code/Chest-xray-landmark-dataset/Images/pred_PCAH/CQ500-CT-228_CT 4cc sec 150cc D3D on-2.stl
saved mesh /media/fmatzkin/data/franco/Code/Chest-xray-landmark-dataset/Images/pred_PCAH/CQ500-CT-328_CT 0.625mm.stl
saved mesh /media/fmatzkin/data/franco/Code/Chest-xray-landmark-dataset/Images/pred_PCAH/CQ500-CT-105_CT I To S.stl
saved mesh /media/fmatzkin/data/franco/Code/Chest-xray-landmark-dataset/Images/pred_PCAH/CQ500-CT-67_CT BONE.stl
saved mesh /media/fmatzkin/dat

In [ ]:
from headctools.tools.offline_augmentation import simulate_cr_images as sim
sim('/home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/some_of_test/',2,)